In [15]:
import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset

# Load your dataset from CSV
df = pd.read_csv('forT5.csv')
df = df[:10000]

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("flax-community/t5-recipe-generation", use_fast=True)

def preprocess_data(example):
    input_text = example['ingredients']
    output_text = example['recepie']
    return tokenizer(input_text, padding="max_length", truncation=True, return_tensors="pt"), \
           tokenizer(output_text, padding="max_length", truncation=True, return_tensors="pt")

# Preprocess the dataset
train_data = [preprocess_data(row) for index, row in df.iterrows()]

# Convert the preprocessed data into a format suitable for Hugging Face's datasets
input_ids = [item[0]["input_ids"].squeeze() for item in train_data]
attention_mask = [item[0]["attention_mask"].squeeze() for item in train_data]
labels = [item[1]["input_ids"].squeeze() for item in train_data]

dataset_dict = {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
}

# Create a Dataset object
dataset = Dataset.from_dict(dataset_dict)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
df

,Unnamed: 0,ingredients,recepie
0,0,"['extra firm tofu', 'eggplant', 'zucchini', 'm...","['Drain the tofu', 'carefully squeezing out ex..."
1,1,"['graham cracker crumbs', 'sugar', 'butter', '...","['Graham Cracker Crust: In small bowl', 'combi..."
2,2,"['carrots', 'eggs', 'white sugar', 'all-purpos...","['Beat together the eggs', 'oil', 'and white s..."
3,3,"['butter', 'almond paste', 'sugar', 'eggs', 'f...","['Preheat oven to 350 degrees Fahrenheit.', 'C..."
4,4,"['egg', 'corn syrup', 'lemon juice', 'vanilla'...","['Preheat oven to 375F.', 'Leave shells in foi..."
...,...,...,...
9995,9995,"['self-rising flour', 'whole wheat flour', 'dr...","['Preheat the oven to 375 degrees.', 'Lightly ..."
9996,9996,"['flour', 'baking powder', 'salt', 'sugar', 'e...",['Preheat oven to 400* F. Grease a 12-cup muff...
9997,9997,"['all-purpose flour', 'whole wheat flour', 'dr...","['Preheat oven to 400 degrees.', 'Combine flou..."
9998,9998,"['water', 'black tea', 'nonfat milk', 'Splenda...",['Heat water to boiling. Add tea bags; reduce ...


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained("flax-community/t5-recipe-generation")


In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduce batch size
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Adjust based on memory
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,  # Limit the number of saved checkpoints
    save_steps=100,  # Save less frequently to reduce overhead
    logging_dir='./logs',
    logging_steps=500,
    dataloader_num_workers=2,  # Adjust number of workers
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

/Users/saikiranpilli/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss
500,2.084900
1000,0.722400
1500,0.697700
2000,0.667700
2500,0.662900
3000,0.655600
3500,0.647300


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=3750, training_loss=0.8625944458007813, metrics={'train_runtime': 33762.58, 'train_samples_per_second': 0.889, 'train_steps_per_second': 0.111, 'total_flos': 1.82687367168e+16, 'train_loss': 0.8625944458007813, 'epoch': 3.0})

In [21]:
trainer.save_model("./trained_model")

In [22]:
print('hi')

hi


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("flax-community/t5-recipe-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("./trained_model")




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
# Example ingredients
input_ingredients = "Milk, rice, egg"

# Tokenize the input
inputs = tokenizer(input_ingredients, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

# Generate the recipe
with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,  # Adjust as necessary
        num_beams=4,  # Number of beams for beam search
        early_stopping=True
    )



In [1]:
# Decode the generated tokens
generated_recipe = tokenizer.decode(output[0], skip_special_tokens=True)


generated_recipe = generated_recipe.split('.')
print("Generated Recipe:")

for i in generated_recipe:
    print(i)

'# Decode the generated tokens\ngenerated_recipe = tokenizer.decode(output[0], skip_special_tokens=True)\n\n\ngenerated_recipe = generated_recipe.split(\'.\')\nprint("Generated Recipe:")\n\nfor i in generated_recipe:\n    print(i)'